In [ ]:
import gradio as gr
from openai import OpenAI
import os
from dotenv import load_dotenv
from pydantic import BaseModel, Field

# --------------------------
# Setup
# --------------------------

load_dotenv()

OPENAI_KEY = os.getenv("OPENAI_API_KEY")
OPENAI_MODEL = os.getenv("OPENAI_MODEL", "gpt-5-nano")

if not OPENAI_KEY:
    raise RuntimeError("OPENAI_API_KEY is not set in the environment")

client = OpenAI(api_key=OPENAI_KEY)

class JobExtraction(BaseModel):
    role: str = Field(..., description="Job title or position name.")
    must_have: list[str] = Field(..., description="Top required skills or qualifications.")
    nice_to_have: list[str] = Field(default_factory=list, description="Optional but desirable skills.")
    responsibilities: str = Field(..., description="Key responsibilities of the role.")

/Users/vitorlaguna/repos/curriclick/ai/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def greet(user_prompt):

    system_prompt = (
        "Extract the role and skills from this job description. "
        "Return ONLY JSON that matches the schema."
        "\n\n<your job text here>"
    )

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt},
    ]

    completion = client.chat.completions.parse(
        model=OPENAI_MODEL,
        messages=messages,
        response_format=JobExtraction,
    )

    return completion.choices[0].message

In [3]:
response = greet("""

Sobre a vaga
Na Stefanini, acreditamos no poder da colaboração. Co-criamos soluções inovadoras em parceria com nossos clientes, combinando tecnologia de ponta, inteligência artificial e a criatividade humana. Estamos na vanguarda da resolução de problemas de negócios, proporcionando impacto real em escala global.
 
Ao se juntar à Stefanini, você se torna parte de uma jornada global de transformação. Estamos empenhados em criar impacto positivo não apenas nos negócios, mas também na vida de nossos colaboradores. Se você procura uma oportunidade de crescimento profissional em uma empresa que valoriza inovação, respeito, autonomia e parceria, você encontra aqui! 



Junte-se a nós e seja parte da mudança!


Estamos em busca de um Desenvolvedor Full Stack para se juntar à nossa equipe. Este profissional será responsável por desenvolver e manter aplicações web, com foco em alta disponibilidade e baixa latência, além de garantir a qualidade e a automatização de testes.


Principais atividades:
- Desenvolvimento e manutenção de aplicações web;
- Implementação das funcionalidades levantadas pelo departamento de análise;
- Integração de sistemas;
- Automação de testes;
- Desenvolvimento de funcionalidades no Back-End;
- Desenvolvimento de funcionalidades no Front-End.


Requisitos:
- Experiência prévia nas linguagens informadas;
- Conhecimento avançado em React, Angular, Node.js, Typescript, PHP e AWS;
- Conhecimento em metodologias ágeis (Scrum/Kanban);
- Inglês Intermediário.


Diferenciais:
- Conhecimentos em Docker e Kubernetes;
- Experiência com TDD e BDD;
- Conhecimento em CI/CD.


Benefícios:


🍛 Vale Alimentação ou Vale Refeição;
👨🏼 🎓 Desconto em cursos, universidades e instituições de idiomas;
📚 Academia Stefanini - plataforma com cursos online, gratuitos, atualizados e com certificado;
🗣 Mentoring;
💉 Clube de vantagens para consultas e exames;
🏥 Assistência Médica;
🦷Assistência Odontológica;
🛫 Clube de viagens;
🐶 Convênio para Pet;
e muito mais...




""")

In [5]:
print(response.content)

{"role":"Desenvolvedor Full Stack","must_have":["React","Angular","Node.js","Typescript","PHP","AWS","Metodologias ágeis (Scrum/Kanban)","Inglês Intermediário"],"nice_to_have":["Docker","Kubernetes","TDD","BDD","CI/CD"],"responsibilities":"Desenvolvimento e manutenção de aplicações web; Implementação das funcionalidades levantadas pelo departamento de análise; Integração de sistemas; Automação de testes; Desenvolvimento de funcionalidades no Back-End; Desenvolvimento de funcionalidades no Front-End."}


## 🧠 Interview Agent

The **Interview Agent** is responsible for collecting and maintaining structured information about a user’s professional profile, ensuring that each interaction gradually enriches its understanding of the candidate.
It acts as the **first step** of the candidate evaluation process, providing a clean, structured summary that feeds directly into the **CVBuilder Agent**.

### 🎯 Core Functionality

When provided with a job description in JSON format — containing fields such as `role`, `must_have`, `nice_to_have`, and `responsibilities` — the Interview Agent:

1. **Analyzes required skills**
   It scans all `must_have` and `nice_to_have` skills listed in the job input.

2. **Consults the local database**
   The agent checks whether it already possesses stored information about the current user, including:

   * Technical skills and proficiency levels
   * Professional experience and key projects
   * Educational background
   * Language proficiency
   * Certifications
   * Contact details (email, phone number)
   * Location (state, city, neighborhood)

3. **Identifies missing information**
   For any skills or attributes not yet known, the agent formulates concise, formal questions in a single response.
   Example:

   ```
   What is your experience with Typescript?
   Have you ever worked with Kubernetes?
   What is your level of expertise with CI/CD?
   ```

4. **Updates the knowledge base**
   After the user provides answers, the agent automatically saves the new data in its local database, ensuring that each piece of information is permanently available for future interviews — eliminating repetitive questioning.

5. **Generates a Fit Summary**
   Once all relevant information is collected, the Interview Agent produces a structured summary detailing how the user’s skills, background, and experience align with the requirements of the given job description.
   This output serves as the input for the **CVBuilder Agent**, which uses it to automatically generate or update the candidate’s résumé.

---

### 🧩 Example Flow

**Input:**

```json
{
  "role": "Desenvolvedor Full Stack",
  "must_have": ["React", "Node.js", "Typescript", "AWS"],
  "nice_to_have": ["Docker", "CI/CD"]
}
```

**System Behavior:**

1. The agent checks which of these technologies it already knows the user has experience with.
2. It asks questions only about missing ones (e.g., Docker and CI/CD).
3. It stores the answers locally.
4. It outputs a **comprehensive fit report**, summarizing the candidate’s technical and professional match for the position.

In [6]:
import sqlite3
import json
from typing import Any, Dict, List, Optional


DB_PATH = os.getenv("INTERVIEW_AGENT_DB", "interview_agent.db")

# ---------------------------
# SQLite: ultra-simple schema
# ---------------------------
# - users(email PK, phone, state, city, neighborhood)
# - profile(email FK, education_json, certifications_json, languages_json, experience_json)
# - skills(email FK, skill, details_json)
#
# "details_json" is a JSON with fields you decide to store, e.g.:
#   {"experience":"2 years building internal dashboards", "level":"Advanced"}
#
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS users (
            email TEXT PRIMARY KEY,
            phone TEXT,
            state TEXT,
            city TEXT,
            neighborhood TEXT
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS profile (
            email TEXT PRIMARY KEY,
            education_json TEXT DEFAULT '[]',
            certifications_json TEXT DEFAULT '[]',
            languages_json TEXT DEFAULT '[]',
            experience_json TEXT DEFAULT '[]',
            FOREIGN KEY(email) REFERENCES users(email)
        )
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS skills (
            email TEXT,
            skill TEXT,
            details_json TEXT,
            PRIMARY KEY (email, skill),
            FOREIGN KEY(email) REFERENCES users(email)
        )
    """)
    conn.commit()
    conn.close()

# ---------------------------
# DB helpers
# ---------------------------
def _get_conn():
    return sqlite3.connect(DB_PATH)

def get_user_core(email: str) -> Dict[str, Optional[str]]:
    conn = _get_conn()
    cur = conn.cursor()
    cur.execute("SELECT email, phone, state, city, neighborhood FROM users WHERE email = ?", (email,))
    row = cur.fetchone()
    conn.close()
    if not row:
        return {"email": email, "phone": None, "state": None, "city": None, "neighborhood": None}
    return {"email": row[0], "phone": row[1], "state": row[2], "city": row[3], "neighborhood": row[4]}

def get_user_profile(email: str) -> Dict[str, Any]:
    conn = _get_conn()
    cur = conn.cursor()
    cur.execute("SELECT education_json, certifications_json, languages_json, experience_json FROM profile WHERE email = ?", (email,))
    row = cur.fetchone()
    conn.close()
    if not row:
        return {"education": [], "certifications": [], "languages": [], "experience": []}
    return {
        "education": json.loads(row[0] or "[]"),
        "certifications": json.loads(row[1] or "[]"),
        "languages": json.loads(row[2] or "[]"),
        "experience": json.loads(row[3] or "[]"),
    }

def get_user_skills(email: str) -> Dict[str, Dict[str, Any]]:
    conn = _get_conn()
    cur = conn.cursor()
    cur.execute("SELECT skill, details_json FROM skills WHERE email = ?", (email,))
    rows = cur.fetchall()
    conn.close()
    return {skill: json.loads(details_json or "{}") for (skill, details_json) in rows}

def upsert_user_core(email: str, phone: Optional[str], state: Optional[str], city: Optional[str], neighborhood: Optional[str]):
    conn = _get_conn()
    cur = conn.cursor()
    cur.execute("INSERT OR IGNORE INTO users(email, phone, state, city, neighborhood) VALUES (?, ?, ?, ?, ?)",
                (email, phone, state, city, neighborhood))
    cur.execute("UPDATE users SET phone=COALESCE(?, phone), state=COALESCE(?, state), city=COALESCE(?, city), neighborhood=COALESCE(?, neighborhood) WHERE email=?",
                (phone, state, city, neighborhood, email))
    conn.commit()
    conn.close()

def upsert_user_profile(email: str, education: Optional[List[Dict]]=None, certifications: Optional[List[Dict]]=None,
                        languages: Optional[List[Dict]]=None, experience: Optional[List[Dict]]=None):
    conn = _get_conn()
    cur = conn.cursor()
    cur.execute("INSERT OR IGNORE INTO profile(email) VALUES (?)", (email,))
    if education is not None:
        cur.execute("UPDATE profile SET education_json=? WHERE email=?", (json.dumps(education), email))
    if certifications is not None:
        cur.execute("UPDATE profile SET certifications_json=? WHERE email=?", (json.dumps(certifications), email))
    if languages is not None:
        cur.execute("UPDATE profile SET languages_json=? WHERE email=?", (json.dumps(languages), email))
    if experience is not None:
        cur.execute("UPDATE profile SET experience_json=? WHERE email=?", (json.dumps(experience), email))
    conn.commit()
    conn.close()

def upsert_user_skill(email: str, skill: str, details: Dict[str, Any]):
    conn = _get_conn()
    cur = conn.cursor()
    cur.execute("INSERT OR REPLACE INTO skills(email, skill, details_json) VALUES (?, ?, ?)",
                (email, skill, json.dumps(details)))
    conn.commit()
    conn.close()

In [ ]:
from agents import function_tool, RunContextWrapper

# ---------------------------
# Tools exposed to the Agent
# ---------------------------

@function_tool
def fetch_profile(ctx: RunContextWrapper[Any], email: str, skills: List[str]) -> Dict[str, Any]:
    """
    Return known user info and which skills are missing.

    Args:
        email: The user's email (primary key).
        skills: The list of skills from the job JSON (must_have + nice_to_have).
    """
    core = get_user_core(email)
    profile = get_user_profile(email)
    known_skills = get_user_skills(email)
    missing_skills = [s for s in skills if s not in known_skills]

    # Which core/profile attributes are missing?
    missing_profile = {
        "phone": core["phone"] is None,
        "state": core["state"] is None,
        "city": core["city"] is None,
        "neighborhood": core["neighborhood"] is None,
        "education": len(profile["education"]) == 0,
        "certifications": len(profile["certifications"]) == 0,
        "languages": len(profile["languages"]) == 0,
        "experience": len(profile["experience"]) == 0,
    }

    return {
        "core": core,
        "profile": profile,
        "known_skills": known_skills,
        "missing_skills": missing_skills,
        "missing_profile": missing_profile,
    }


@function_tool
def save_profile_facts(
    ctx: RunContextWrapper[Any],
    email: str,
    phone: Optional[str] = None,
    state: Optional[str] = None,
    city: Optional[str] = None,
    neighborhood: Optional[str] = None,
    education: Optional[List[str]] = None,        # ← was List[Dict[str, Any]]
    certifications: Optional[List[str]] = None,   # ←
    languages: Optional[List[str]] = None,        # ←
    experience: Optional[List[str]] = None,       # ←
) -> str:
    upsert_user_core(email, phone, state, city, neighborhood)
    upsert_user_profile(email, education, certifications, languages, experience)
    return "User profile facts saved."


@function_tool
def save_skill_details(ctx: RunContextWrapper[Any], email: str, skill: str, experience_text: str, level: Optional[str] = None) -> str:
    """
    Upsert a single skill for the user.

    Args:
        email: User email.
        skill: Skill name (e.g., 'React').
        experience_text: Short description of hands-on experience.
        level: Optional level label (Beginner/Intermediate/Advanced/Expert).
    """
    upsert_user_skill(email, skill, {"experience": experience_text, "level": level})
    return f"Saved skill {skill}."

UserError: additionalProperties should not be set for object types. This could be because you're using an older version of Pydantic, or because you configured additional properties to be allowed. If you really need this, update the function or output tool to not use a strict schema.

In [ ]:
# Initialize local DB once
if "INTERVIEW_DB_INIT" not in globals():
    try:
        init_db()
    except NameError:
        # If init_db is not defined in this kernel, ignore (import path case)
        pass
    INTERVIEW_DB_INIT = True

In [ ]:
from agents import Agent, Runner, RunConfig

# ---------------------------
# The Interview Agent
# ---------------------------
INSTRUCTIONS = """
You are the Interview Agent.

Goals:
1) Read a job JSON with fields: role, must_have[], nice_to_have[], responsibilities.
2) Fetch known user data via tools, using the provided email.
3) If any skills or profile attributes are missing, ask concise, formal questions for ALL missing items in one single message. Do not repeat questions already answered in DB.
   - For skills: ask one question per missing skill, e.g., "What is your experience with Docker?" Optionally ask for a level (Beginner/Intermediate/Advanced/Expert).
   - For profile: ask for only the missing attributes among: phone, state, city, neighborhood, education, certifications, languages, experience.
4) After the user answers, call the save tools to persist all new facts and skills.
5) Finally, produce a clear "Fit Summary" that maps the user's known capabilities to the job's must_have and nice_to_have, plus a short narrative summary (3–5 lines). Keep tone formal and concise.
6) The final output should be suitable as input for a downstream CVBuilderAgent.

Output policy:
- If information is missing, output ONLY the consolidated questions block.
- If nothing is missing (or after saving new info), output the Fit Summary only.

IMPORTANT:
- Always call fetch_profile first.
- Never ask for data that is already stored.
- Keep questions minimal and formal.
- Never include implementation details about the tools.
"""

agent = Agent(
    name="InterviewAgent",
    instructions=INSTRUCTIONS,
    tools=[fetch_profile, save_profile_facts, save_skill_details],
    # You can override the model globally when running with Runner if you like.
)

# ---------------------------
# Simple runner helpers
# ---------------------------
def build_first_turn_prompt(email: str, job_json: Dict[str, Any]) -> str:
    """
    Builds the first instruction to the agent.
    """
    must = job_json.get("must_have", [])
    nice = job_json.get("nice_to_have", [])
    skills = must + nice

    return (
        "You will receive a job JSON and a user email.\n"
        f"User email: {email}\n"
        f"Job JSON:\n{json.dumps(job_json, ensure_ascii=False, indent=2)}\n\n"
        f"Combined skills list for fetch_profile: {json.dumps(skills, ensure_ascii=False)}\n"
        "Proceed."
    )

def run_interview(email: str, job_json: Dict[str, Any], model: Optional[str] = None):
    prompt = build_first_turn_prompt(email, job_json)
    run_config = RunConfig(model=model) if model else None
    result = Runner.run_sync(agent, prompt, run_config=run_config)
    return result.final_output

In [ ]:
import traceback

EXAMPLE_JOB = {
    "role": "Desenvolvedor Full Stack",
    "must_have": [
        "React",
        "Angular",
        "Node.js",
        "Typescript",
        "PHP",
        "AWS",
        "Metodologias ágeis (Scrum/Kanban)",
        "Inglês Intermediário"
    ],
    "nice_to_have": ["Docker", "Kubernetes", "TDD", "BDD", "CI/CD"],
    "responsibilities": "Desenvolvimento e manutenção de aplicações web; Implementação das funcionalidades levantadas; Integração de sistemas; Automação de testes; Back-End e Front-End."
}

def run_turn1(email: str, job_json_text: str, model_name: str):
    """
    Turn 1: Build the first-turn prompt and get the agent output.
    If information is missing, the agent will output ONLY the consolidated questions block.
    """
    if not email.strip():
        return "❗ Please provide an email."

    # Parse job JSON
    try:
        job = json.loads(job_json_text)
    except Exception as e:
        return f"❗ Invalid JSON:\n```\n{e}\n```"

    try:
        # If run_interview is available, use it (it wraps first turn)
        out = run_interview(email, job, model=model_name or None)
        return out
    except NameError:
        # Fallback: run explicitly via Runner with first-turn prompt
        try:
            prompt = build_first_turn_prompt(email, job)
        except Exception as e:
            return f"❗ Could not build first-turn prompt:\n```\n{e}\n```"

        try:
            rc = RunConfig(model=model_name) if model_name else None
            result = Runner.run_sync(agent, prompt, run_config=rc)
            return result.final_output
        except Exception as e:
            return "❗ Error running the agent:\n```\n" + traceback.format_exc() + "\n```"
    except Exception as e:
        return "❗ Error running the agent:\n```\n" + traceback.format_exc() + "\n```"

def run_turn2(user_answers: str, model_name: str):
    """
    Turn 2: Paste the user's answers (free text). The agent will call save_* tools
    and then produce the Fit Summary.
    """
    if not user_answers.strip():
        return "❗ Please paste your answers."

    try:
        rc = RunConfig(model=model_name) if model_name else None
        result = Runner.run_sync(agent, user_answers, run_config=rc)
        return result.final_output
    except Exception as e:
        return "❗ Error running the agent on Turn 2:\n```\n" + traceback.format_exc() + "\n```"

with gr.Blocks(title="Interview Agent — Minimal Runner") as demo:
    gr.Markdown("## Interview Agent (Turn-based Test)\nUse Turn 1 to get the questions. Then paste answers into Turn 2 to save and receive the Fit Summary.")

    with gr.Row():
        email = gr.Textbox(label="User Email (PK)", value="candidate@example.com", scale=1)
        model = gr.Textbox(label="Model (optional)", value=OPENAI_MODEL, scale=1)

    job_json = gr.Textbox(
        label="Job JSON",
        value=json.dumps(EXAMPLE_JOB, ensure_ascii=False, indent=2),
        lines=16
    )

    turn1_btn = gr.Button("▶️ Run Turn 1 (Inspect & Ask Missing)")
    turn1_out = gr.Markdown(label="Turn 1 Output")

    gr.Markdown("---")

    answers = gr.Textbox(
        label="Turn 2 — Paste your answers here (free text). Example:",
        value=(
            "Phone: +55 11 99999-0000\n"
            "City/State/Neighborhood: São Paulo, SP — Vila Mariana\n"
            "Education: Bacharel em Sistemas de Informação (USP, 2026)\n"
            "Languages: Português (Nativo), Inglês (Intermediário B2)\n"
            "Certifications: AWS Certified Cloud Practitioner (2024)\n"
            "Experience: 2 anos Full Stack (React/Node/Typescript), 1 ano AWS\n"
            "Docker: 1 ano — Intermediário\n"
            "CI/CD: 1 ano — Intermediário (GitHub Actions)\n"
            "Kubernetes: Sem experiência prática\n"
        ),
        lines=12
    )
    turn2_btn = gr.Button("💾 Run Turn 2 (Save & Fit Summary)")
    turn2_out = gr.Markdown(label="Turn 2 Output")

    # Wire events
    turn1_btn.click(
        fn=run_turn1,
        inputs=[email, job_json, model],
        outputs=[turn1_out],
    )

    turn2_btn.click(
        fn=run_turn2,
        inputs=[answers, model],
        outputs=[turn2_out],
    )

# In Jupyter: inline=True renders directly in the notebook
demo.launch(inline=True, share=False)